In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/loan-approval-classification-data/loan_data.csv


In [2]:
df = pd.read_csv('/kaggle/input/loan-approval-classification-data/loan_data.csv')
df.head(5)

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1


# Data Analysis

In [3]:
df.shape

(45000, 14)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   person_age                      45000 non-null  float64
 1   person_gender                   45000 non-null  object 
 2   person_education                45000 non-null  object 
 3   person_income                   45000 non-null  float64
 4   person_emp_exp                  45000 non-null  int64  
 5   person_home_ownership           45000 non-null  object 
 6   loan_amnt                       45000 non-null  float64
 7   loan_intent                     45000 non-null  object 
 8   loan_int_rate                   45000 non-null  float64
 9   loan_percent_income             45000 non-null  float64
 10  cb_person_cred_hist_length      45000 non-null  float64
 11  credit_score                    45000 non-null  int64  
 12  previous_loan_defaults_on_file  

In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
person_age,45000.0,27.764178,6.045108,20.00,24.00,26.00,30.00,144.00
person_income,45000.0,80319.053222,80422.498632,8000.00,47204.00,67048.00,95789.25,7200766.00
person_emp_exp,45000.0,5.410333,6.063532,0.00,1.00,4.00,8.00,125.00
loan_amnt,45000.0,9583.157556,6314.886691,500.00,5000.00,8000.00,12237.25,35000.00
loan_int_rate,45000.0,11.006606,2.978808,5.42,8.59,11.01,12.99,20.00
loan_percent_income,45000.0,0.139725,0.087212,0.00,0.07,0.12,0.19,0.66
cb_person_cred_hist_length,45000.0,5.867489,3.879702,2.00,3.00,4.00,8.00,30.00
credit_score,45000.0,632.608756,50.435865,390.00,601.00,640.00,670.00,850.00
loan_status,45000.0,0.222222,0.415744,0.00,0.00,0.00,0.00,1.00


### Conclusion:
- data needs to be scaled

In [6]:
y = df['loan_status']
df= df.drop(columns=['loan_status'])
df.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No


x,y separated

In [7]:
df.shape

(45000, 13)

# Preprocessing

In [8]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [9]:
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
non_numeric_cols = df.select_dtypes(exclude=['int64', 'float64']).columns

In [10]:
scaler = StandardScaler()
scaled_numeric = scaler.fit_transform(df[numeric_cols])
scaled_numeric_df = pd.DataFrame(scaled_numeric, columns=numeric_cols)

In [11]:
encoder = LabelEncoder()
encoded_non_numeric = df[non_numeric_cols].apply(encoder.fit_transform)

In [12]:
final_df = pd.concat([scaled_numeric_df, encoded_non_numeric], axis=1)
final_df.head()

,person_age,person_income,person_emp_exp,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,person_gender,person_education,person_home_ownership,loan_intent,previous_loan_defaults_on_file
0,-0.953538,-0.104090,-0.892284,4.024953,1.683039,4.016394,-0.739109,-1.419814,0,4,3,4,0
1,-1.118963,-0.846005,-0.892284,-1.359209,0.044782,-0.684829,-0.996863,-2.549975,0,3,2,1,1
2,-0.457264,-0.844065,-0.397517,-0.646600,0.625557,3.443074,-0.739109,0.047412,0,3,0,3,0
3,-0.788113,-0.007039,-0.892284,4.024953,1.417829,3.443074,-0.996863,0.840507,0,1,3,3,0
4,-0.622689,-0.176371,-0.727362,4.024953,1.095549,4.475050,-0.481354,-0.924130,1,4,3,3,0


# Model Building

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
x_train,x_test,y_train, y_test = train_test_split(final_df,y,test_size=0.2,random_state=4)

In [15]:
x_train.head(5)

,person_age,person_income,person_emp_exp,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,person_gender,person_education,person_home_ownership,loan_intent,previous_loan_defaults_on_file
8725,-0.953538,-0.091556,-0.892284,-0.892054,-0.119716,-1.028821,-0.996863,-0.210344,0,4,0,1,0
5013,-0.622689,-0.345816,-0.727362,-0.052758,0.706127,0.461810,-0.996863,0.265513,0,1,0,0,0
25039,1.031560,0.210950,1.416627,0.224368,-0.005575,-0.340837,0.034155,0.206031,1,4,0,2,1
3398,-0.788113,-0.164745,-0.562440,0.224368,0.001139,0.232483,-0.996863,0.959472,1,3,3,3,1
36295,0.866135,-0.602476,-0.067673,-1.200852,0.424132,-0.914157,0.549665,-1.043094,0,3,0,2,0


In [16]:
x_train.shape

(36000, 13)

In [17]:
x_test.shape

(9000, 13)

In [18]:
y_train.shape

(36000,)

### import libraries

In [19]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [21]:
import kerastuner as kt

## Hyperparameter Tuning

In [22]:
def build_model(hp):
    model = Sequential()
    counter = 0
    for i in range(hp.Int('num_layers', min_value = 1, max_value=10)):
        if counter == 0:
            model.add(Dense(hp.Int('units'+str(i), min_value=8, max_value=128, step=8),
                           activation = hp.Choice('activation', values=['relu', 'tanh', 'leaky_relu']),
                           input_dim=13))
            model.add(Dropout(hp.Choice('dropout'+str(i), values= [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        else:
            model.add(Dense(hp.Int('units'+str(i), min_value=8, max_value=128, step=8),
                           activation = hp.Choice('activation', values=['relu', 'tanh', 'leaky_relu'])))
            model.add(Dropout(hp.Choice('dropout'+str(i), values= [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        counter +=1
    model.add(Dense(1,activation='sigmoid')) 
    model.compile(optimizer=hp.Choice('optimizer', values=['rmsprop', 'adam', 'sgd']),
                                      loss='binary_crossentropy', metrics=['accuracy'])
    return model
           

In [23]:
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=3)
tuner = kt.RandomSearch(hypermodel=build_model, objective='val_accuracy', max_trials=5, directory='mydir')
tuner.search(x_train,y_train, epochs=50, validation_split=0.2)

Trial 5 Complete [00h 01m 48s]
val_accuracy: 0.8787500262260437

Best val_accuracy So Far: 0.9087499976158142
Total elapsed time: 00h 09m 05s


In [27]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2,
 'units0': 120,
 'activation': 'leaky_relu',
 'dropout0': 0.4,
 'optimizer': 'adam',
 'units1': 128,
 'dropout1': 0.9,
 'units2': 120,
 'dropout2': 0.1,
 'units3': 88,
 'dropout3': 0.7,
 'units4': 72,
 'dropout4': 0.3,
 'units5': 40,
 'dropout5': 0.8,
 'units6': 72,
 'dropout6': 0.5,
 'units7': 40,
 'dropout7': 0.9,
 'units8': 24,
 'dropout8': 0.6,
 'units9': 112,
 'dropout9': 0.3}

In [29]:
model_final = tuner.get_best_models(num_models=1)[0]

In [32]:
model_final.fit(x_train, y_train, epochs=200, initial_epoch=50,validation_split=0.2, callbacks=callback)

Epoch 51/200
900/900 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9018 - loss: 0.2198 - val_accuracy: 0.9036 - val_loss: 0.2089
Epoch 52/200
900/900 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9036 - loss: 0.2138 - val_accuracy: 0.9076 - val_loss: 0.2088
Epoch 53/200
900/900 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9026 - loss: 0.2161 - val_accuracy: 0.9061 - val_loss: 0.2084
Epoch 54/200
900/900 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9071 - loss: 0.2131 - val_accuracy: 0.9054 - val_loss: 0.2077
Epoch 55/200
900/900 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9039 - loss: 0.2128 - val_accuracy: 0.9057 - val_loss: 0.2095
Epoch 56/200
900/900 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9052 - loss: 0.2149 - val_accuracy: 0.9062 - val_loss: 0.2096
Epoch 57/200
900/900 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9022 - loss: 0.2191 - val_accuracy: 0.9054 - val_loss: 0.2070
Epoch 58/200
900/900 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8990 - loss: 0.2231 - 

In [36]:
model_final.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 120)                 │           1,680 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 120)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          15,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 51,893 (202.71 KB)

 Trainable params: 17,297 (67.57 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 34,596 (135.14 KB)

In [34]:
#y_pred = model_final.predict(x_test)

282/282 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


# Testing on unseen data

In [39]:
loss, accuracy = model_final.evaluate(x_test, y_test, verbose=0)
print(accuracy)

0.913444459438324


In [40]:
print(loss)

0.19333674013614655


### conclusion
- accuracy on unseen data is 0.9134